# 감성분류를 위한 코드

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00


In [5]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00


In [6]:
# 파이토치 라이브러리로, 딥러닝 모델을 만들고 학습하는 데 사용
import torch
# Hugging face dadasets 라이브러리로, 다양한 데이터셋을 쉽게 로드
from datasets import load_dataset
# Hugging face transformers 라이브러리로, 다양한 트랜스포머 모델을 로드 및 사용
from transformers import(
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
import pandas as pd
import numpy as np
# 모델 평가를 위한 라이브러리
import evaluate

In [ ]:
# 데이터 셋 로드
dataset = load_dataset('sepidmnorozy/Korean_sentiment')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.csv:   0%|          | 0.00/3.29M [00:00<?, ?B/s]

dev.csv:   0%|          | 0.00/127k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/239k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/36000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1333 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2667 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 36000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1333
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 2667
    })
})

In [ ]:
# 데이터셋 출력
print(dataset['train'][3118])
print(dataset['train'][14310])

{'label': 1, 'text': '졸잼!!!성아가나중에억울한일이잇어서좀슬펏는데마지막은기쁘게끝나서다행이에여'}
{'label': 0, 'text': '진짜 어떻게 된놈의 영화가 여고괴담 1보다도 못함? 신기하다 그것도 2012년작이 1998년보다 못함 솔까 여고괴담1은 반전은 최고지 뭐 이놈의 영화는 여고괴담 시리즈보다도 못하는거같다'}


# 토큰화 Tokenize
한국어 감성 분석 모델을 훈련시키기 위한 데이터 전처리 및 준비 과정을 수행

In [ ]:
# 사용할 모델의 이름을 지정
model_name = 'kykim/bert-kor-base'

In [ ]:
# 한국어 텍스트 처리를 위한 BERT 모델
# 지정한 모델에 맞는 토크나이저를 로드
# 토크나이저는 텍스트 데이터를 BERT 모델이 처리할 수 있는 형식으로 변환하는 도구
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer
# 토크나이저의 설정 및 정보를 출력

# >> 위 코드를 실행하면 지정된 모델에 해당하는 사전 학습된 토크나이저를 로드

# AutoTokenizer
## Hugging Face의 Transformers 라이브러리에서 제공하는 클래스
## 제공된 모델 이름에 맞게 토크나이저 클래스를 자동으로 선택하고 로드
## 사전 학습된 토크나이저 로드
## 여러 모델에 대해 동일한 인터페이스를 제공하므로, 다양한 모델을 사용할 때 코드의 일관성 유지 가능

# from_pretrained
## 경로에 맞는 사전 학습된 토크나이저를 로드

tokenizer_config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BertTokenizerFast(name_or_path='kykim/bert-kor-base', vocab_size=42000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
# 토크나이저 함수 정의
def tokenizer_func(x):
    return tokenizer(
        x['text'],
        padding='max_length', # 모든 텍스트를 최재 길이에 맞추어 패딩
        max_length=256,       # 최대 길이를 256으로 설정, 모델이 처리할 수 있는 입력의 최대 길이
        truncation=True       # 텍스트가 최대 길이를 초과할 경우 잘라내는 옵션
    )

# 로드된 토크나이저를 사용하여 입력 텍스트를 BERT 모델이 처리할 수 있는 형식으로 변환

In [ ]:
# 데이터셋의 각 샘플에 tokenizer_func를 적용 >> 토큰화된 데이터를 생성
tokenized_datasets = dataset.map(tokenizer_func, batched=True)

# map
## datasets 라이브러리에서 제공하는 강력한 기능으로,
## 데이터셋의 각 샘플에 대해 지정된 함수를 적용

# batched=True
## map 함수가 데이터를 샘플 단위가 아닌 배치 단위로 처리하도록 함
## 여러 샘플을 한 번에 처리하기 때문에 속도가 크게 향상

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1333 [00:00<?, ? examples/s]

Map:   0%|          | 0/2667 [00:00<?, ? examples/s]

In [ ]:
# 훈련데이터셋 샘플 수
train_num_samples = 10000

# train_num_samples 만큼의 샘플을 선택
# 모델 훈련 속도를 높이기 위해 훈련 데이터의 크기를 줄이는 데 사용
train_ds = tokenized_datasets['train'].shuffle(seed=42).select(range(train_num_samples))
eval_ds = tokenized_datasets['validation'].shuffle(seed=42)

# 전이학습 Transfer Learning
사전 학습된 모델을 새로운 작업에 재사용하여 학습 시간을 줄이고 성능을 향상시킴



---
### 사전학습(pre-training)

: 모델을 대규모 데이터셋에서 일반적인 특성을 학습하도록 훈련

### 미세조정(fine-tuning)

: 사전학습된 모델을 새로운 특정 작업에 맞게 조정


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
# AutoModelForSequenceClassification 클래스는 특정 시퀸스 분류 작업을 수행하도록 설계된 모델을 자동 로드
# from_pretrained : 지정된 모델 이름을 기반으로 사전 학습된 가중치를 모델에 로드
# num_labels=2 : 예측할 클래스(레이블)의 수를 지정

# 전이학습
## 허깅페이스 모델 허브에 사전 학습된 모델 가중치를 다운로드하고 로드
## 모델이 특정 작업에 맞게 설정(시퀀스 분류)

pytorch_model.bin:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at kykim/bert-kor-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# HyperParams

In [ ]:
bs = 32       # 한 번의 훈련 단계에서 사용하는 데이터 샘플의 수
## 배치 크기가 클수록 한 번의 훈련단계에서 더 많은 샘플을 사용하지만, 메모리 사용량 증가
epochs = 4    # 전체 데이터셋을 몇 번 반복하여 훈련할지
lr = 1e-5     # 학습률로 모델이 가중치를 업데이트 하는 속도 조절
## 학습률이 너무 높으면 모델이 최적화되지 않을 수 있고, 너무 낮으면 학습이 너무 오래 걸릴 수 있음

In [ ]:
# 훈련 인자
args = TrainingArguments(
    'outputs',
    learning_rate = lr,
    warmup_ratio = 0.1,
    lr_scheduler_type = 'cosine',
    fp16 = True,
    evaluation_strategy='epoch',
    per_device_train_batch_size = bs,
    per_device_eval_batch_size = bs,
    gradient_accumulation_steps=4, #until bs =128
    eval_accumulation_steps = 4,
    num_train_epochs=epochs,
    weight_decay=0.01,
    report_to = 'none'
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Metrics

In [ ]:
# 성능 평가 지표
metric = evaluate.load('accuracy')

# all Transformers models retrun logits
def compute_metrics(eval_pred):
    logits, labels =eval_pred
    preds = np.argmax(logits, axis=-1)
    return metric.compute(predictions=preds, references=labels)

# 모델 예측값과 실제 라벨을 받아 정확도를 계산하는 함수

In [ ]:
class CustomTrainer(Trainer):
    def _save_checkpoint(self, model, trial, metrics=None):
        # 모델을 저장하기 전에 모든 텐서를 contiguous로 만듦
        for name, param in model.named_parameters():
            if param is not None:
                param.data = param.data.contiguous()
                if param.grad is not None:
                    param.grad.data = param.grad.data.contiguous()
        super()._save_checkpoint(model, trial, metrics)

# Trainer

In [ ]:
# 커스텀 트레이너 설정
# 모델, 훈련인자, 훈련 데이터셋, 평가 데이터셋, 토크나이저, 성능평가함수
trainer = CustomTrainer(
    model,
    args,
    train_dataset=train_ds, # 훈련데이터셋
    eval_dataset=eval_ds,   # 평가데이터셋
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

# Validation Loss : 모델이 학습하지 않은 데이터에 대해 얼마나 잘 예측하는지를 측정
##  loss 값이 감소하면 모델이 점점 더 나은 성능을 보여줌
##  증가 또는 일정하게 유지하면, 모델이 과적합되고 있을 가능성
##  >> 하이퍼파라미터 조정이나 데이터 전처리, 모델 구조의 변경 등을 고려

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,0.333990,0.867217
1,No log,0.305374,0.879970
2,No log,0.288292,0.885971
3,No log,0.286165,0.888972


TrainOutput(global_step=312, training_loss=0.3454272196843074, metrics={'train_runtime': 442.357, 'train_samples_per_second': 90.425, 'train_steps_per_second': 0.705, 'total_flos': 5247486888099840.0, 'train_loss': 0.3454272196843074, 'epoch': 3.987220447284345})

In [ ]:
trainer.save_model('/content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/code/Crawling/지우/mymodels')

# 추론 Inference

In [7]:
pipe = pipeline('text-classification', model='/content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/code/EDA/예진/sentiment_model')
# 파이프라인 주요 구성요소
## 모델 : 사전 훈련된 모델이나 fine tuned된 모델
## 토크나이저 : 입력 텍스트를 모델이 이해할 수 있는 형태로 변환한 도구
## 모델 : 저장된 모델의 경로를 지정, 이 경로에서 모델을 로드하여 텍스트 분류 작업 수행

# pipline은 Hugging Face Transformers 라이브러리에서 제공하는 API, 특정 작업을 쉽게 수행

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [15]:
import pandas as pd
path = '/content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/code/Crawling/예진/data/df2_exp_review_gray_keyword.csv'
raw = pd.read_csv(path)
df = raw.copy()

In [16]:
df.head()

,name,address,NAVER_name,category,review_num,reviews,이용방법,대기시간,방문목적,동행자
0,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객이...,예약 없이 이용,10분 이내,여행,"연인・배우자, 아이"
1,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",오픈런 했고 대기 1번이라서 사람이 많이 없을거라 생각했는데 식당 안에 손님이 가득...,예약 후 이용,10분 이내,여행,연인・배우자
2,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 편...,예약 후 이용,30분 이내,여행,"연인・배우자, 아이"
3,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","뼈없는, 가시없는 갈치조림먹으려고 찾아보고왔어요!\n순살갈치라니ㅋㅋ 먹기너무편하고 ...",예약 없이 이용,30분 이내,여행,연인・배우자
4,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","제주 올때마다 가보고 싶었는데 시간이 안맞기도 했고, 재료소진일 때도 있어서 늘 실...",예약 후 이용,1시간 이상,"데이트, 여행",연인・배우자


In [17]:
df.shape

(167315, 10)

In [19]:
df['reviews'].isnull().sum()

1905

In [20]:
df['reviews'].dropna(inplace=True)

In [21]:
df.shape

(167315, 10)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167315 entries, 0 to 167314
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   name        167315 non-null  object
 1   address     167315 non-null  object
 2   NAVER_name  167315 non-null  object
 3   category    167315 non-null  object
 4   review_num  167315 non-null  object
 5   reviews     165410 non-null  object
 6   이용방법        77619 non-null   object
 7   대기시간        77690 non-null   object
 8   방문목적        75560 non-null   object
 9   동행자         74402 non-null   object
dtypes: object(10)
memory usage: 12.8+ MB


In [23]:
df['reviews'] = df['reviews'].astype(str)

In [25]:
df['reviews']

,reviews
0,제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객이...
1,오픈런 했고 대기 1번이라서 사람이 많이 없을거라 생각했는데 식당 안에 손님이 가득...
2,순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 편...
3,"뼈없는, 가시없는 갈치조림먹으려고 찾아보고왔어요!\n순살갈치라니ㅋㅋ 먹기너무편하고 ..."
4,"제주 올때마다 가보고 싶었는데 시간이 안맞기도 했고, 재료소진일 때도 있어서 늘 실..."
...,...
167310,사진 찍을 겨를도 없이 맛있게 먹었네요^^.\n사장님이 친절하게 대해주셔서 자주 오...
167311,월남쌈 맛있어요
167312,맛있어요
167313,"양이 많아요,, 배불러서 좀 남겼는데 생각나네요ㅠ\n편백찜 + 샤브 + 칼국수 + ..."


In [27]:
# 정규화
import re
df['reviews'].apply(lambda x : re.sub(r'[^ㄱ-ㅣ가-힣]+', ' ', x))

,reviews
0,제주도 여행 하면서 처음 먹어본 갈치조림 순살이라서 더 먹기 편했어요 여행객이 많지...
1,오픈런 했고 대기 번이라서 사람이 많이 없을거라 생각했는데 식당 안에 손님이 가득 ...
2,순살 갈치 조림이 궁금해서 방문 했습니다 현장 테이블링만 가능하고 회전은 빠른 편이...
3,뼈없는 가시없는 갈치조림먹으려고 찾아보고왔어요 순살갈치라니ㅋㅋ 먹기너무편하고 젓가락...
4,제주 올때마다 가보고 싶었는데 시간이 안맞기도 했고 재료소진일 때도 있어서 늘 실패...
...,...
167310,사진 찍을 겨를도 없이 맛있게 먹었네요 사장님이 친절하게 대해주셔서 자주 오고싶습니...
167311,월남쌈 맛있어요
167312,맛있어요
167313,양이 많아요 배불러서 좀 남겼는데 생각나네요ㅠ 편백찜 샤브 칼국수 죽 모두 좋았습니다


In [28]:
# 데이터프레임을 데이터셋으로 변환
from datasets import Dataset

dataset = Dataset.from_pandas(df)
# Dataset 객체는 모델 훈련 및 평가 과정에서 사용하기 적합한 형식으로,
# 데이터 셋의 샘플을 빠르고 효율적으로 처리할 수 있도록 지원

In [29]:
dataset

Dataset({
    features: ['name', 'address', 'NAVER_name', 'category', 'review_num', 'reviews', '이용방법', '대기시간', '방문목적', '동행자'],
    num_rows: 167315
})

In [30]:
# 모델 이름과 토크나이저 초기화
model_name = 'kykim/bert-kor-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [33]:
# 토크나이저 함수 정의
def tokenizer_func(examples):
    return tokenizer(
        examples['reviews'],
        padding='max_length',
        max_length=256,
        truncation=True
    )

In [34]:
tokenized_review = dataset.map(tokenizer_func, batched=True)
# 데이터셋의 각 항목에 tokenizer_func을 적용하여 토크나이즈를 함
# batched=True는 배치 단위로 처리하여 속도를 높임

# tokenized_review에는 토크나이저를 사용하여 각 텍스트 샘플을 토큰화한 결과

Map:   0%|          | 0/167315 [00:00<?, ? examples/s]

In [36]:
# 데이터셋 크기 확인
dataset_size = len(tokenized_review)
dataset_size
# train_num_samples = min(10000, dataset_size)  # 데이터셋 크기를 넘지 않도록 설정

167315

In [37]:
df = tokenized_review.select(range(dataset_size))

# tokenized_review에는 토크나이저를 사용하여 각 텍스트 샘플을 토큰화한 결과
# select 메서드는 특정 샘플을 선택하거나 필터링할 때 사용

In [38]:
df

Dataset({
    features: ['name', 'address', 'NAVER_name', 'category', 'review_num', 'reviews', '이용방법', '대기시간', '방문목적', '동행자', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 167315
})

In [39]:
preds_txts = pipe(df['reviews'])

In [40]:
preds_txts_df = pd.DataFrame(preds_txts)

In [41]:
preds_txts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167315 entries, 0 to 167314
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   label   167315 non-null  object 
 1   score   167315 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.6+ MB


In [42]:
preds_txts_df.head()

,label,score
0,LABEL_1,0.948258
1,LABEL_1,0.881020
2,LABEL_0,0.972762
3,LABEL_1,0.977383
4,LABEL_1,0.560018


In [43]:
preds_txts_df['label'] = preds_txts_df['label'].map({'LABEL_1':1, 'LABEL_0':0})

In [44]:
review_labeling_df = pd.DataFrame(df)

In [45]:
review_labeling_df.head()

,name,address,NAVER_name,category,review_num,reviews,이용방법,대기시간,방문목적,동행자,input_ids,token_type_ids,attention_mask
0,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객이...,예약 없이 이용,10분 이내,여행,"연인・배우자, 아이","[2, 18616, 14437, 15042, 14121, 24882, 29078, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",오픈런 했고 대기 1번이라서 사람이 많이 없을거라 생각했는데 식당 안에 손님이 가득...,예약 후 이용,10분 이내,여행,연인・배우자,"[2, 15415, 8595, 17911, 17663, 18960, 14972, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 편...,예약 후 이용,30분 이내,여행,"연인・배우자, 아이","[2, 5103, 8487, 29078, 40115, 8018, 25331, 145...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","뼈없는, 가시없는 갈치조림먹으려고 찾아보고왔어요!\n순살갈치라니ㅋㅋ 먹기너무편하고 ...",예약 없이 이용,30분 이내,여행,연인・배우자,"[2, 4758, 15075, 2014, 21325, 15075, 29078, 20...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","제주 올때마다 가보고 싶었는데 시간이 안맞기도 했고, 재료소진일 때도 있어서 늘 실...",예약 후 이용,1시간 이상,"데이트, 여행",연인・배우자,"[2, 15148, 5691, 17404, 32233, 17854, 14629, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [49]:
review_labeling_df.drop(columns=['input_ids', 'token_type_ids', 'attention_mask'], inplace=True)

In [50]:
preds_txts_df = pd.concat([preds_txts_df, review_labeling_df], axis=1)
preds_txts_df

,label,score,name,address,NAVER_name,category,review_num,reviews,이용방법,대기시간,...,name,address,NAVER_name,category,review_num,reviews,이용방법,대기시간,방문목적,동행자
0,1,0.948258,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객이...,예약 없이 이용,10분 이내,...,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객이...,예약 없이 이용,10분 이내,여행,"연인・배우자, 아이"
1,1,0.881020,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",오픈런 했고 대기 1번이라서 사람이 많이 없을거라 생각했는데 식당 안에 손님이 가득...,예약 후 이용,10분 이내,...,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",오픈런 했고 대기 1번이라서 사람이 많이 없을거라 생각했는데 식당 안에 손님이 가득...,예약 후 이용,10분 이내,여행,연인・배우자
2,0,0.972762,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 편...,예약 후 이용,30분 이내,...,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 편...,예약 후 이용,30분 이내,여행,"연인・배우자, 아이"
3,1,0.977383,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","뼈없는, 가시없는 갈치조림먹으려고 찾아보고왔어요!\n순살갈치라니ㅋㅋ 먹기너무편하고 ...",예약 없이 이용,30분 이내,...,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","뼈없는, 가시없는 갈치조림먹으려고 찾아보고왔어요!\n순살갈치라니ㅋㅋ 먹기너무편하고 ...",예약 없이 이용,30분 이내,여행,연인・배우자
4,1,0.560018,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","제주 올때마다 가보고 싶었는데 시간이 안맞기도 했고, 재료소진일 때도 있어서 늘 실...",예약 후 이용,1시간 이상,...,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","제주 올때마다 가보고 싶었는데 시간이 안맞기도 했고, 재료소진일 때도 있어서 늘 실...",예약 후 이용,1시간 이상,"데이트, 여행",연인・배우자
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167310,1,0.985755,일공공샤브,제주 서귀포시 대정읍 구억리 835-4번지 1층,일공공샤브&편백찜 영어교육도시점,일공공키친,30,사진 찍을 겨를도 없이 맛있게 먹었네요^^.\n사장님이 친절하게 대해주셔서 자주 오...,예약 없이 이용,바로 입장,...,일공공샤브,제주 서귀포시 대정읍 구억리 835-4번지 1층,일공공샤브&편백찜 영어교육도시점,일공공키친,30,사진 찍을 겨를도 없이 맛있게 먹었네요^^.\n사장님이 친절하게 대해주셔서 자주 오...,예약 없이 이용,바로 입장,나들이,부모님
167311,1,0.810249,일공공샤브,제주 서귀포시 대정읍 구억리 835-4번지 1층,일공공샤브&편백찜 영어교육도시점,일공공키친,30,월남쌈 맛있어요,예약 후 이용,바로 입장,...,일공공샤브,제주 서귀포시 대정읍 구억리 835-4번지 1층,일공공샤브&편백찜 영어교육도시점,일공공키친,30,월남쌈 맛있어요,예약 후 이용,바로 입장,친목,연인・배우자
167312,1,0.868907,일공공샤브,제주 서귀포시 대정읍 구억리 835-4번지 1층,일공공샤브&편백찜 영어교육도시점,일공공키친,30,맛있어요,예약 후 이용,바로 입장,...,일공공샤브,제주 서귀포시 대정읍 구억리 835-4번지 1층,일공공샤브&편백찜 영어교육도시점,일공공키친,30,맛있어요,예약 후 이용,바로 입장,"친목, 회식",지인・동료
167313,1,0.979986,일공공샤브,제주 서귀포시 대정읍 구억리 835-4번지 1층,일공공샤브&편백찜 영어교육도시점,일공공키친,30,"양이 많아요,, 배불러서 좀 남겼는데 생각나네요ㅠ\n편백찜 + 샤브 + 칼국수 + ...",예약 후 이용,None,...,일공공샤브,제주 서귀포시 대정읍 구억리 835-4번지 1층,일공공샤브&편백찜 영어교육도시점,일공공키친,30,"양이 많아요,, 배불러서 좀 남겼는데 생각나네요ㅠ\n편백찜 + 샤브 + 칼국수 + ...",예약 후 이용,None,None,None


In [51]:
preds_txts_df[preds_txts_df.label == 0]

,label,score,name,address,NAVER_name,category,review_num,reviews,이용방법,대기시간,...,name,address,NAVER_name,category,review_num,reviews,이용방법,대기시간,방문목적,동행자
2,0,0.972762,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 편...,예약 후 이용,30분 이내,...,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 편...,예약 후 이용,30분 이내,여행,"연인・배우자, 아이"
36,0,0.625347,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","금요일 오후 1시, 웨이팅 15팀 있었고\n30분정도 대기했습니다!\n무뼈갈치조림,...",None,None,...,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","금요일 오후 1시, 웨이팅 15팀 있었고\n30분정도 대기했습니다!\n무뼈갈치조림,...",None,None,None,None
100,0,0.994410,번영가든,제주 제주시 봉개동 1385번지 1층,번영가든,한식,14,식당에 들어서는 순간 가성비가 좋을 것 같은 기대감 상승해서 기분 좋아짐.\n골고루...,None,None,...,번영가든,제주 제주시 봉개동 1385번지 1층,번영가든,한식,14,식당에 들어서는 순간 가성비가 좋을 것 같은 기대감 상승해서 기분 좋아짐.\n골고루...,None,None,None,None
126,0,0.813003,번영김밥,제주 제주시 봉개동 1902번지,번영김밥,종합분식,40,아침 간식,None,None,...,번영김밥,제주 제주시 봉개동 1902번지,번영김밥,종합분식,40,아침 간식,None,None,None,None
127,0,0.520685,번영김밥,제주 제주시 봉개동 1902번지,번영김밥,종합분식,40,여전히 일가는 도중에 들려서 몇줄 샀어요.,None,None,...,번영김밥,제주 제주시 봉개동 1902번지,번영김밥,종합분식,40,여전히 일가는 도중에 들려서 몇줄 샀어요.,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167254,0,0.969083,인화계,제주 제주시 일도이동 379-19번지 1층,인화계,닭갈비,46,추천받아서 갔는데 간이 제 입에는 좀 짰어요,예약 없이 이용,바로 입장,...,인화계,제주 제주시 일도이동 379-19번지 1층,인화계,닭갈비,46,추천받아서 갔는데 간이 제 입에는 좀 짰어요,예약 없이 이용,바로 입장,데이트,연인・배우자
167261,0,0.598111,인화계,제주 제주시 일도이동 379-19번지 1층,인화계,닭갈비,46,닭갈비는 맛있어요~ 찌개는 좀...짠듯\n계란찜..막국수는 괜찮아요~,None,None,...,인화계,제주 제주시 일도이동 379-19번지 1층,인화계,닭갈비,46,닭갈비는 맛있어요~ 찌개는 좀...짠듯\n계란찜..막국수는 괜찮아요~,None,None,None,None
167265,0,0.931491,인화계,제주 제주시 일도이동 379-19번지 1층,인화계,닭갈비,46,...,None,None,...,인화계,제주 제주시 일도이동 379-19번지 1층,인화계,닭갈비,46,...,None,None,None,None
167292,0,0.994131,일공공샤브,제주 서귀포시 대정읍 구억리 835-4번지 1층,일공공샤브&편백찜 영어교육도시점,일공공키친,30,"음식의 맛을 떠나서, 직원분의 실수에 의한 대처가 상당히 불쾌했습니다. 주문 실수로...",예약 없이 이용,바로 입장,...,일공공샤브,제주 서귀포시 대정읍 구억리 835-4번지 1층,일공공샤브&편백찜 영어교육도시점,일공공키친,30,"음식의 맛을 떠나서, 직원분의 실수에 의한 대처가 상당히 불쾌했습니다. 주문 실수로...",예약 없이 이용,바로 입장,여행,연인・배우자


In [52]:
preds_txts_df.to_csv('/content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/code/EDA/예진/data/review_sentiment.csv', index=0)

In [53]:
path = '/content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/code/EDA/예진/data/review_sentiment.csv'
df = pd.read_csv(path)

In [ ]:

# 같은 행에서 df['label']의 값이 1이면 df['review']에 [긍정]이라는 텍스트를 추가
df.loc[df['label'] == 1, 'review'] = '[긍정] ' + df['review']

# 같은 행에서 df['label']의 값이 0이면 df['review']에 [부정]이라는 텍스트를 추가
df.loc[df['label'] == 0, 'review'] = '[부정] ' + df['review']

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10039 entries, 0 to 10038
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0.2       10039 non-null  int64  
 1   label              10039 non-null  int64  
 2   score              10039 non-null  float64
 3   Unnamed: 0.1       10039 non-null  int64  
 4   Unnamed: 0         10039 non-null  int64  
 5   name               10039 non-null  object 
 6   address            10039 non-null  object 
 7   NAVER_name         10039 non-null  object 
 8   category           10039 non-null  object 
 9   review_num         10039 non-null  object 
 10  reviews            10039 non-null  object 
 11  review             10039 non-null  object 
 12  __index_level_0__  10039 non-null  int64  
 13  input_ids          10039 non-null  object 
 14  token_type_ids     10039 non-null  object 
 15  attention_mask     10039 non-null  object 
dtypes: float64(1), int64(5